# Mittelfluss aus Investitionstätigkeit

## Einlesen der erforderlichen Paket und Daten

Auch fpr die Rekonstruktion des Mittelflusses aus Investitionstätigkeit
muss als Erstes die pandas Library importiert werden. Anschliessend 
müssen die Daten einem pandas DataFrame zugewiesen werden.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv', index_col=0)

## Rekonstruktion des Mittelflusses aus Investitionstätigkeit

Für die Rekonstruktion des Mittelflusses aus Investitionstätigkeit kann auf die
'Formel' zurückgegriffen werden, welche im 
[Schweizerischen Kontenrahmen KMU](https://www.kmu.admin.ch/dam/kmu/de/dokumente/savoir-pratique/Finances/240812%20Schulkontenrahmen%20VEB%20-%20DE.pdf.download.pdf/240812%20Schulkontenrahmen%20VEB%20-%20DE.pdf)
abgedruckt ist. Die 'Formel' ist unten aufgeführt.

$–$ Investitionen in Finanzanlagen  
$+$ Devestitionen von Finanzanlagen  
$–$ Investitionen in Beteiligungen  
$+$ Devestitionen von Beteiligungen  
$–$ Investitionen in mobile Sachanlagen  
$+$ Devestitionen von mobilen Sachanlagen  
$–$ Investitionen in immobile Sachanlagen  
$+$ Devestitionen von immobilen Sachanlagen  
$–$ Investitionen in immaterielle Werte  
$+$ Devestitionen von immateriellen Werten  

Die meisten Schulaufgaben gehen allerdings von bloss zwei Positionen im Anlagevermögen aus:
Mobilien und Immobilien. Aus diesem Grund wird im vorliegenden Beispiel
nur mit den Positionen Mobilien und Immobilien gearbeitet. Die zu Grunde
liegenden Überlegungen sind jedoch auch auf die anderen Positionen anwendbar.

### Rekonstruktion der Abschreibungen auf Mobilien

Die Nettoinvestitionen in Mobilien können berechnet werden, indem die
Veränderung der Mobilien während der Beobachtungsperiode zu den
Abschreibungen auf Mobilien addiert wird.

#### Definition der erforderlichen Helferfunktionen

Helferfunktionen werden in Python mit einem einleitenden Unterstrich definiert.

##### `_get_total_absch`

Um die Abschreibungen auf einzelnen Anlagen zu berechnen, muss zuerst
das Total der Abschreibungen ermittelt werden. Dies ist Aufgabe der Funktion
`_get_total_absch`.

In [3]:
def _get_total_absch(ledger: pd.DataFrame) -> float:
    """
    Berechnet die Summe aller Abschreibungen aus dem Hauptbuch.

    Diese Funktion summiert alle Abschreibungen und Wertberichtigungen auf Positionen
    des Anlagevermögens aus den Konten 6800-6899 gemäss Kontenrahmen VEB. Die Werte
    werden mit -1 multipliziert, da Aufwände im Hauptbuch als negative Werte geführt
    werden.

    Args:
        ledger (pd.DataFrame): DataFrame mit Hauptbuchdaten. Muss einen Index mit
            Kontonummern und eine Spalte 'Abschluss' mit den Schlusssaldi enthalten.

    Returns:
        float: Summe aller Abschreibungen als positiver Wert.

    Example:
        >>> hauptbuch = pd.DataFrame({
        ...     'Abschluss': [-1000, -500]
        ... }, index=[6800, 6810])
        >>> _get_total_absch(hauptbuch)
        1500.0
    """
    # Liste aller möglichen Abschreibungskonten (6800-6899) erstellen
    konti = [i for i in range(6800, 6900)]
    
    # Initialisierung der Summe aller Abschreibungen
    total_absch = 0
    
    # Iteration über alle möglichen Abschreibungskonten
    for konto in konti:
        # Prüfen ob das Konto im Hauptbuch existiert
        if konto in ledger.index:
            # Abschlusssaldo mit -1 multiplizieren um aus dem negativen Aufwand
            # einen positiven Wert zu machen
            total_absch += ledger.loc[konto, 'Abschluss'] * -1
    
    return total_absch

##### Kontrolle, ob Wertberichtigungskonti vorliegen

###### `_check_wb_mob`

Die Funktion `_check_wb_mob` überprüft, ob es Wertberichtigungskonti für
die Mobilien gibt. Die Werberichtigungskonti für Mobile Sachanlagen
verteilen sich auf mehrere Konti.

In [4]:
def _check_wb_mob(ledger: pd.DataFrame) -> bool:
    """
    Prüft, ob Wertberichtigungskonten für mobile Sachanlagen im Hauptbuch vorhanden sind.

    Diese Funktion untersucht das Hauptbuch auf die Existenz von Wertberichtigungskonten
    (WB-Konten) für mobile Sachanlagen gemäss Kontenrahmen VEB. Die Prüfung umfasst
    folgende Konten:
    - 1509: WB Maschinen und Apparate
    - 1519: WB Mobiliar und Einrichtungen
    - 1529: WB Büromaschinen, Informatik, Kommunikationstechnologie
    - 1539: WB Fahrzeuge
    - 1549: WB Werkzeuge und Geräte

    Args:
        ledger (pd.DataFrame): DataFrame mit Hauptbuchdaten. Der Index muss die
            Kontonummern enthalten.

    Returns:
        bool: True, wenn mindestens ein WB-Konto für mobile Sachanlagen existiert,
              False, wenn keine WB-Konten gefunden wurden.

    Example:
        >>> hauptbuch = pd.DataFrame(index=[1510, 1519, 1520])
        >>> _check_wb_mob(hauptbuch)
        True  # Weil Konto 1519 (WB Mobiliar) vorhanden ist
    """
    # Schnittmenge zwischen Hauptbuchkonten und WB-Konten für mobile Sachanlagen bilden
    # Dies gibt uns alle existierenden WB-Konten zurück
    wb_mob = ledger.index.intersection([1509, 1519, 1529, 1539, 1549])
    
    # True zurückgeben, wenn mindestens ein WB-Konto gefunden wurde (Länge > 0)
    return len(wb_mob) > 0

###### `_check_wb_immo`

Die Funktion `_check_wb_immo` überprüft, ob es Wertberichtigungskonti für
die Immobilien gibt. Die Funktionalität dieser Funktion könnte auch
direkt in der Funktion, welche sie aufruft, implementiert werden. Aus
Gründen der Symmetrie erhält sie jedoch eine eigene Funktion - analog
zur Funktion `_check_wb_mob`.

In [5]:
def _check_wb_immo(ledger: pd.DataFrame) -> bool:
    """
    Prüft, ob Wertberichtigungskonten für Geschäftsliegenschaften im Hauptbuch vorhanden sind.

    Diese Funktion untersucht das Hauptbuch auf die Existenz des Wertberichtigungskontos
    (WB-Konto) für Geschäftsliegenschaften gemäss Kontenrahmen VEB. Die Prüfung umfasst
    das Konto:
    - 1609: Wertberichtigungen Geschäftsliegenschaften (WB für Konto 1600)

    Die Funktion wird im Rahmen der Investitionsrechnung verwendet, um die korrekte
    Berechnungsmethode für Investitionen in Geschäftsliegenschaften zu bestimmen.

    Args:
        ledger (pd.DataFrame): DataFrame mit Hauptbuchdaten. Der Index muss die
            Kontonummern gemäss Kontenrahmen VEB enthalten.

    Returns:
        bool: True, wenn das WB-Konto 1609 für Geschäftsliegenschaften existiert,
              False, wenn kein entsprechendes WB-Konto gefunden wurde.

    Example:
        >>> hauptbuch = pd.DataFrame(index=[1600, 1609, 1700])
        >>> _check_wb_immo(hauptbuch)
        True  # Weil Konto 1609 (WB Geschäftsliegenschaften) vorhanden ist
    """
    # Prüfen ob Konto 1609 (Wertberichtigungen Geschäftsliegenschaften) im Index
    # des Hauptbuchs existiert
    return 1609 in ledger.index

##### `_get_delta_wb_mob`

In [6]:
def _get_delta_wb_mob(ledger: pd.DataFrame) -> float:
    """
    Berechnet die Veränderung der Wertberichtigungen für mobile Sachanlagen.

    Diese Funktion ermittelt die Veränderung (Delta) der Wertberichtigungen für
    mobile Sachanlagen durch Vergleich der Eröffnungs- und Abschlusssaldi.
    Folgende Wertberichtigungskonten werden berücksichtigt:
    - 1509: WB Maschinen und Apparate
    - 1519: WB Mobiliar und Einrichtungen
    - 1529: WB Büromaschinen, Informatik, Kommunikationstechnologie
    - 1539: WB Fahrzeuge
    - 1549: WB Werkzeuge und Geräte

    Args:
        ledger (pd.DataFrame): DataFrame mit Hauptbuchdaten. Muss einen Index mit
            Kontonummern sowie die Spalten 'Eroeffnung' und 'Abschluss' enthalten.

    Returns:
        float: Gesamtveränderung der Wertberichtigungen für mobile Sachanlagen.
               Gibt 0 zurück, wenn keine WB-Konten gefunden wurden.

    Example:
        >>> hauptbuch = pd.DataFrame({
        ...     'Eroeffnung': [-1000, -500],
        ...     'Abschluss': [-1200, -600]
        ... }, index=[1509, 1519])
        >>> _get_delta_wb_mob(hauptbuch)
        -300.0  # Zunahme der WB um 300 (negative Werte sind Wertminderungen)
    """
    # Prüfen ob WB-Konten für mobile Sachanlagen existieren
    if not _check_wb_mob(ledger):
        return 0
    else:
        # Ermitteln der vorhandenen WB-Konten durch Schnittmenge mit möglichen Konten
        wb_mob = ledger.index.intersection([1509, 1519, 1529, 1539, 1549])
        
        # Initialisierung der Summe der WB-Veränderungen
        delta_wb_mob = 0
        
        # Berechnung der Veränderung für jedes gefundene WB-Konto
        # Delta = Abschlusssaldo - Eröffnungssaldo
        for konto in wb_mob:
            delta_wb_mob += ledger.loc[konto, 'Abschluss'] - ledger.loc[konto, 'Eroeffnung']
    
    # Rückgabe der Gesamtveränderung der Wertberichtigungen
    return delta_wb_mob * -1

##### `_get_delta_wb_immo`

In [7]:
def _get_delta_wb_immo(ledger: pd.DataFrame) -> float:
    """
    Berechnet die Veränderung der Wertberichtigungen für Geschäftsliegenschaften.

    Diese Funktion ermittelt die Veränderung (Delta) der Wertberichtigungen für
    Geschäftsliegenschaften durch Vergleich der Eröffnungs- und Abschlusssaldi.
    Berücksichtigt wird das Konto:
    - 1609: Wertberichtigungen Geschäftsliegenschaften (WB für Konto 1600)

    Die Funktion ist Teil der Investitionsrechnung und wird zur Ermittlung der
    Netto-Investitionen in Geschäftsliegenschaften verwendet.

    Args:
        ledger (pd.DataFrame): DataFrame mit Hauptbuchdaten. Muss einen Index mit
            Kontonummern sowie die Spalten 'Eroeffnung' und 'Abschluss' enthalten.

    Returns:
        float: Gesamtveränderung der Wertberichtigungen für Geschäftsliegenschaften.
               Gibt 0 zurück, wenn kein WB-Konto (1609) gefunden wurde.

    Example:
        >>> hauptbuch = pd.DataFrame({
        ...     'Eroeffnung': [-1000],
        ...     'Abschluss': [-1200]
        ... }, index=[1609])
        >>> _get_delta_wb_immo(hauptbuch)
        -200.0  # Zunahme der WB um 200 (negative Werte sind Wertminderungen)
    """
    # Prüfen ob WB-Konto für Geschäftsliegenschaften (1609) existiert
    if not _check_wb_immo(ledger):
        return 0
    else:
        # Berechnung der Veränderung: Abschlusssaldo - Eröffnungssaldo
        delta_wb_immo = ledger.loc[1609, 'Abschluss'] - ledger.loc[1609, 'Eroeffnung']
        return delta_wb_immo * -1


##### `_get_absch_mob`

In [8]:
def _get_absch_mob(ledger: pd.DataFrame) -> float:
    """
    Berechnet die Abschreibungen auf mobile Sachanlagen (Mobilien).

    Diese Funktion ermittelt die Abschreibungen auf mobile Sachanlagen basierend auf
    den Wertberichtigungskonten (WB-Konten) oder, falls diese nicht vorhanden sind,
    als Differenz zwischen Gesamtabschreibungen und Immobilien-Abschreibungen.

    Die Berechnung erfolgt in zwei möglichen Varianten:
    1. Bei vorhandenen WB-Konten für Mobilien (1509-1549):
       - Direkte Berechnung aus der Veränderung der WB-Konten
    2. Bei fehlenden WB-Konten für Mobilien:
       - Berechnung als Differenz zwischen Gesamtabschreibungen und
         Immobilien-Abschreibungen

    Args:
        ledger (pd.DataFrame): DataFrame mit Hauptbuchdaten. Muss einen Index mit
            Kontonummern sowie die Spalten 'Eroeffnung' und 'Abschluss' enthalten.

    Returns:
        float: Abschreibungen auf mobile Sachanlagen für die Periode.

    Example:
        >>> hauptbuch = pd.DataFrame({
        ...     'Eroeffnung': [-1000],
        ...     'Abschluss': [-1200]
        ... }, index=[1519])
        >>> _get_absch_mob(hauptbuch)
        200.0  # Zunahme der WB um 200 entspricht Abschreibung von 200
    """
    # Ermittlung der Gesamtabschreibungen aus Konto 6800
    absch = _get_total_absch(ledger)    
 
    # Prüfung ob WB-Konten für Mobilien existieren (1509-1549)
    if _check_wb_mob(ledger):
        # Falls WB-Konten existieren: Abschreibungen = Veränderung der WB-Konten
        absch_mob = _get_delta_wb_mob(ledger)
    else:
        # Falls keine WB-Konten für Mobilien: Berechnung über Immobilien-WB
        delta_wb_immo = _get_delta_wb_immo(ledger)
        print("Delta WB Immobilien:", delta_wb_immo)
        if delta_wb_immo < 0:
            # Bei negativem Delta (= Zunahme WB Immobilien):
            # Alle Abschreibungen betreffen Mobilien
            absch_mob = absch
        else:
            # Bei positivem Delta (= Abnahme WB Immobilien):
            # Mobilien-Abschreibungen = Gesamtabschreibungen - Immobilien-WB-Änderung
            absch_mob = absch - delta_wb_immo
        
    return absch_mob


##### `_get_absch_immo`

In [9]:
def _get_absch_immo(ledger: pd.DataFrame) -> float:
    """
    Berechnet die Abschreibungen auf Geschäftsliegenschaften (Immobilien).

    Diese Funktion ermittelt die Abschreibungen auf Geschäftsliegenschaften entweder
    direkt aus der Veränderung des WB-Kontos 1609 oder, falls dieses nicht existiert,
    als Differenz zwischen Gesamtabschreibungen und Mobilien-Abschreibungen.

    Die Berechnung erfolgt in zwei möglichen Varianten:
    1. Bei vorhandenem WB-Konto (1609):
       - Direkte Berechnung aus der Veränderung des WB-Kontos
    2. Bei fehlendem WB-Konto:
       - Berechnung als Differenz zwischen Gesamtabschreibungen (6800-6899)
         und Mobilien-Abschreibungen

    Args:
        ledger (pd.DataFrame): DataFrame mit Hauptbuchdaten. Muss einen Index mit
            Kontonummern sowie die Spalten 'Eroeffnung' und 'Abschluss' enthalten.

    Returns:
        float: Abschreibungen auf Geschäftsliegenschaften für die Periode.
              Ein positiver Wert bedeutet eine Wertminderung.

    Example:
        >>> hauptbuch = pd.DataFrame({
        ...     'Eroeffnung': [-1000],
        ...     'Abschluss': [-1200]
        ... }, index=[1609])
        >>> _get_absch_immo(hauptbuch)
        200.0  # Zunahme der WB um 200 entspricht Abschreibung von 200
    """
    # Prüfen ob WB-Konto für Immobilien (1609) existiert
    if _check_wb_immo(ledger):
        # Falls WB-Konto existiert: Abschreibungen = Veränderung der WB
        # Ein positives Delta bedeutet eine Zunahme der Abschreibungen
        absch_immo = _get_delta_wb_immo(ledger)
    else:
        # Falls kein WB-Konto existiert: Berechnung über Differenzmethode
        # Gesamtabschreibungen aus Konten 6800-6899 ermitteln
        absch = _get_total_absch(ledger)
                
        # Mobilien-Abschreibungen berechnen
        absch_mob = _get_absch_mob(ledger)
                        
        # Immobilien-Abschreibungen = Gesamtabschreibungen - Mobilien-Abschreibungen
        absch_immo = absch + absch_mob    
    
    return absch_immo


##### `_get_invest_mob`

In [10]:
def _get_invest_mob(ledger: pd.DataFrame) -> float:
    """
    Berechnet die Nettoinvestitionen in mobile Sachanlagen für die Periode.

    Diese Funktion ermittelt die Investitionen in mobile Sachanlagen durch Vergleich
    der Eröffnungs- und Abschlusssaldi der relevanten Konten. Folgende Konten werden
    berücksichtigt:
    - 1500: Maschinen und Apparate
    - 1510: Mobiliar und Einrichtungen
    - 1520: Büromaschinen, Informatik, Kommunikationstechnologie
    - 1530: Fahrzeuge
    - 1540: Werkzeuge und Geräte

    Die Berechnung erfolgt in zwei Schritten:
    1. Ermittlung der Bestandesveränderungen der mobilen Sachanlagen
    2. Bei fehlenden Wertberichtigungskonten: Addition der Abschreibungen

    Args:
        ledger (pd.DataFrame): DataFrame mit Hauptbuchdaten. Muss einen Index mit
            Kontonummern sowie die Spalten 'Eroeffnung' und 'Abschluss' enthalten.

    Returns:
        float: Nettoinvestitionen in mobile Sachanlagen für die Periode.
              Ein positiver Wert bedeutet eine Nettoinvestition.

    Example:
        >>> hauptbuch = pd.DataFrame({
        ...     'Eroeffnung': [50, -20],
        ...     'Abschluss': [70, 25]
        ... }, index=[1510, 1519])
        >>> _get_invest_mob(hauptbuch)
        65.0  # Nettoinvestition von 65 (20 Bestandesänderung + 45 Abschreibung)
    """
    # Ermittlung der vorhandenen Konten für mobile Sachanlagen durch
    # Schnittmengenbildung mit möglichen Konten (1500-1540)
    mob = ledger.index.intersection([1500, 1510, 1520, 1530, 1540])
    
    # Initialisierung der Investitionssumme
    invest_mob = 0
    
    # Berechnung der Bestandesveränderungen für jedes gefundene Konto
    # durch Differenz zwischen Abschluss- und Eröffnungssaldo
    for m in mob:
        invest_mob += ledger.loc[m, 'Abschluss'] - ledger.loc[m, 'Eroeffnung']
        
    # Falls keine Wertberichtigungskonten (15x9) vorhanden sind,
    # werden die Abschreibungen zur Investitionssumme addiert
    if not _check_wb_mob(ledger):
        invest_mob += _get_absch_mob(ledger)
        
    return invest_mob


#### `_get_invest_immo`

In [11]:
def _get_invest_immo(ledger: pd.DataFrame) -> float:
    """
    Berechnet die Nettoinvestitionen in Geschäftsliegenschaften für die Periode.

    Diese Funktion ermittelt die tatsächlichen Investitionen in Geschäftsliegenschaften
    (Konto 1600) durch Berücksichtigung der Bestandesveränderung und der Abschreibungen.
    Die Berechnung erfolgt in zwei Schritten:
    1. Ermittlung der Bestandesveränderung (Abschluss - Eröffnung)
    2. Bereinigung um die Abschreibungen

    Wenn die negative Bestandesveränderung exakt den Abschreibungen entspricht,
    haben keine tatsächlichen Investitionen stattgefunden (Rückgabe 0).
    Andernfalls wird die Nettoinvestition als Summe aus Bestandesveränderung
    und Abschreibungen zurückgegeben.

    Args:
        ledger (pd.DataFrame): DataFrame mit Hauptbuchdaten. Muss einen Index mit
            Kontonummern sowie die Spalten 'Eroeffnung' und 'Abschluss' enthalten.

    Returns:
        float: Nettoinvestitionen in Geschäftsliegenschaften.
               Gibt 0 zurück, wenn die Bestandesveränderung nur auf Abschreibungen
               zurückzuführen ist.

    Example:
        >>> hauptbuch = pd.DataFrame({
        ...     'Eroeffnung': [350],
        ...     'Abschluss': [340]
        ... }, index=[1600])
        >>> _get_invest_immo(hauptbuch)
        -10.0  # Negative Nettoinvestition (Desinvestition) von 10
    """
    # Berechnung der Bestandesveränderung der Geschäftsliegenschaften (Konto 1600)
    # durch Differenz zwischen Abschluss- und Eröffnungssaldo
    detla_immo = ledger.loc[1600, 'Abschluss'] - ledger.loc[1600, 'Eroeffnung']
    
    # Ermittlung der Abschreibungen auf Geschäftsliegenschaften
    # mittels Hilfsfunktion _get_absch_immo
    absch_immo = _get_absch_immo(ledger)
        
    # Prüfung ob die Bestandesveränderung nur auf Abschreibungen zurückzuführen ist
    # (negative Bestandesveränderung entspricht den Abschreibungen)
    if detla_immo * -1 == absch_immo:
        return 0
    else:
        # Berechnung der tatsächlichen Nettoinvestition durch Addition
        # der Bestandesveränderung und der Abschreibungen
        return detla_immo + absch_immo


### Funktion `investitionsbereich()`

Die Funktion `investitionsbereich()` ist die Zusammenfassung aller vorher
definierten Hilfsfunktionen. Mit Ihr wird der Mittelfluss aus
Investitionstätigkeit aus dem aufbereiteten DataFrame extrahiert und
formatiert ausgegeben.

In [12]:
def investitionsbereich(ledger: pd.DataFrame):
    """
    Erstellt einen formatierten Bericht über den Mittelfluss aus Investitionen.

    Diese Funktion berechnet und formatiert die Investitionen in mobile und immobile
    Sachanlagen sowie deren Nettosumme. Der Bericht wird mit einer festen Breite von
    50 Zeichen und zentrierter Ausrichtung erstellt.

    Die Berechnung erfolgt in zwei Schritten:
    1. Ermittlung der Investitionen in:
       - Mobile Sachanlagen (Mobilien): Maschinen, Mobiliar, Fahrzeuge etc.
       - Immobile Sachanlagen (Immobilien): Geschäftsliegenschaften
    2. Berechnung der Nettoinvestitionen als Summe beider Kategorien

    Args:
        ledger (pd.DataFrame): DataFrame mit Hauptbuchdaten. Muss die für die
            Investitionsberechnung notwendigen Konten gemäss Kontenrahmen VEB
            enthalten.

    Returns:
        None: Die Funktion gibt den formatierten Bericht direkt auf der Konsole aus.

    Example:
        >>> hauptbuch = pd.DataFrame(...)
        >>> investitionsbereich(hauptbuch)
                    Mittelfluss aus Investitionen

                         Mobilien                 150000
                       Immobilien                 250000
                  Nettoinvestitionen             400000
    """
    # Dictionary für die Investitionswerte initialisieren
    investitionen = {}
    # Investitionen in mobile Sachanlagen berechnen (Konten 1500-1549)
    investitionen['Mobilien'] = _get_invest_mob(ledger)
    # Investitionen in Geschäftsliegenschaften berechnen (Konto 1600)
    investitionen['Immobilien'] = _get_invest_immo(ledger)
    
    # Fixe Breite für die Formatierung des Berichts festlegen
    breite = 50
    # Überschrift des Berichts mit Leerzeilen
    titel = 'Mittelfluss aus Investitionen\n\n'
    
    # Formatierung der einzelnen Investitionspositionen
    for index, value in investitionen.items():
        # Berechnung des verfügbaren Platzes für die Zentrierung
        mitte = breite - len(index) -len(str(value))
        # Berechnung der linken und rechten Breite für die Ausrichtung
        breite_links = len(index) + mitte // 2
        breite_rechts = len(str(value)) + mitte // 2
        # Korrektur bei ungerader Gesamtbreite
        if breite_links + breite_rechts < breite:
            breite_links += 1
        # Formatierte Zeile zum Bericht hinzufügen
        titel += f"{index:<{breite_links}}{value:>{breite_rechts}}\n"
        
    # Berechnung und Formatierung der Nettoinvestitionen
    mitte = breite - len('Nettoinvestitionen') - len(str(investitionen['Mobilien'] + investitionen['Immobilien']))
    breite_links = len('Nettoinvestitionen') + mitte // 2
    breite_rechts = len(str(investitionen['Mobilien'] + investitionen['Immobilien'])) + mitte // 2
    # Korrektur bei ungerader Gesamtbreite
    if breite_links + breite_rechts < breite:
        breite_links += 1
    # Vorbereitung der Nettoinvestitionszeile
    inv = 'Nettoinvestitionen'
    inv_zahl = str(investitionen['Mobilien'] + investitionen['Immobilien'])
    # Formatierte Nettoinvestitionszeile zum Bericht hinzufügen
    titel += f"{inv:<{breite_links}}{inv_zahl:>{breite_rechts}}\n"
        
    # Ausgabe des formatierten Berichts
    print(titel)


In [13]:
investitionsbereich(df)

Mittelfluss aus Investitionen

Mobilien                                      20.0
Immobilien                                    10.0
Nettoinvestitionen                            30.0

